# Preprocessing SMD Appointment Dataset
This jupyter notebook handles the cleaning of the dataset. This includes dropping of rows with null values, inconsistent values, and foreign key violations if any.

The purpose of preprocessing and cleaning the source dataset is to ensure a smooth flow in the ETL process.

# Preloading the source data

In [19]:
import pandas as pd
import warnings
import dask.dataframe as dd
warnings.filterwarnings('ignore')

clinics = pd.read_csv("clinics.csv", header=0, encoding='unicode_escape')
doctors = pd.read_csv("doctors.csv", header=0, encoding='unicode_escape', dtype={'age': str})
px = pd.read_csv("px.csv", header=0, encoding='unicode_escape', dtype={'age': str})

df_clinics = clinics.copy()
df_doctors = doctors.copy()
df_px = px.copy()

In [20]:
appointment_files = [f'split_{i}.csv' for i in range(1, 11)]

# Read the files into a Dask DataFrame
df_appointment = dd.concat([dd.read_csv(f,header=0, encoding='unicode_escape', dtype = str) for f in appointment_files], ignore_index=True)

# Cleaning the clinics table
The clinics table has the following attributes:
- clinicid
- hospitalname
- IsHospital
- City
- Province
- RegionName
  
Let us see clinics that have missing or duplicate values on any attribute and determine if they should be dropped

In [21]:
clinics_with_missing_values = df_clinics[df_clinics.isnull().any(axis=1)]
clinics_with_missing_values

,clinicid,hospitalname,IsHospital,City,Province,RegionName
4,205C3608ECB984C1F5F5D2F52C934428,NaN,False,Burgos,Ilocos Sur,Ilocos Region (I)
7,07FF46BB6597A4F81EED4F59360FF835,NaN,False,Manila,Manila,National Capital Region (NCR)
8,23E582AD8087F2C03A5A31C125123F9A,NaN,False,Manila,Manila,National Capital Region (NCR)
9,5291822D0636DC429E80E953C58B6A76,NaN,False,Makati,Manila,National Capital Region (NCR)
10,588E343066CF54EC3DB5132231DF7D68,NaN,False,Manila,Manila,National Capital Region (NCR)
...,...,...,...,...,...,...
53956,5FD56C1FA3E5C045AC028E4EAB875EB0,NaN,False,Manila,Manila,National Capital Region (NCR)
53957,8DE279A56DBCECE9F9FFC514A7D5A378,NaN,False,Manila,Manila,National Capital Region (NCR)
53958,55A64961C9AA4134016786AE7202682E,NaN,False,Manila,Manila,National Capital Region (NCR)
53959,075E464A7D15E6E5B9D8F8F5B5B16BB9,NaN,False,Manila,Manila,National Capital Region (NCR)


It seems that the majority of null values come from the *hospitalname* attribute. And interestingly, its null when the *isHospital* value is false. While the *hospitalname* is an interesting attribute, it is not justifiable to drop values due to it. The most important attribute for this dimensional table is all related to geographic data. Since all values related to geographic attribute is complete, the only cleaning to be done here is removing duplicate data if any exists.

In [22]:
df_clinics.drop_duplicates(subset=['clinicid'], inplace=True)
df_clinics = df_clinics[df_clinics['clinicid'].isin(df_appointment['clinicid'])]

The *isHospital* values will also be changed accordingly. Instead of *True* or *False*. A tinyint value of 1 or 0 will be used to save space and to ensure valid data type when the ETL script processes the csv.

In [23]:
df_clinics['IsHospital'] = df_clinics['IsHospital'].map({True: 1, False: 0})

# Cleaning the px table
The clinics table has the following attributes:
- pxid
- age
- gender
  
Let us see px (patient) data that have missing values on any attribute and determine if they should be dropped

In [24]:
px_with_missing_values = df_px[df_px.isnull().any(axis=1)]
px_with_missing_values

,pxid,age,gender
0,5A1718EC380AFE6BE24D63EE78CDA043,NaN,FEMALE
150509,F4925C087C35FBC2F685EA6C6E577A17,NaN,FEMALE
150510,5B810EC6EDFA1818DC338923646A58D9,NaN,FEMALE
150511,924A1081F5753DE0770A40BC4E702095,NaN,FEMALE
150512,5CA5A3467106D314655996D182AEA163,NaN,FEMALE
...,...,...,...
4206625,093F00B0A69DA40087DF42799C5C4BBD,NaN,FEMALE
4206662,63E00B95D4ACE351E7949ED8113FAE18,NaN,FEMALE
4206694,52F2A6519C2529AAB9480B1AA5A8C4E9,NaN,FEMALE
4206730,6EAF318B180CD4E9F121C99B5E238914,NaN,FEMALE


A significant amount of missing data is related to the age value. While this is a justifiable cause to drop data. For the purpose of our OLAP application, missing age values are okay as the application is focused more on whether the patient is linked to an appointment. Thus, the only cleaning done here is removing duplicate data if any.

In [25]:
df_px.drop_duplicates(subset=['pxid'], inplace=True)
df_px = df_px[df_px['pxid'].isin(df_appointment['pxid'])]

# Cleaning the doctors table
The clinics table has the following attributes:
- doctorid
- mainspecialty
- age
  
Let us see doctor data that have missing values on any attribute and determine if they should be dropped

In [26]:
doctors_with_missing_values = df_doctors[df_doctors.isnull().any(axis=1)]
doctors_with_missing_values

,doctorid,mainspecialty,age
24,ED3D2C21991E3BEF5E069713AF9FA6CA,NaN,NaN
28,EC8956637A99787BD197EACD77ACCE5E,NaN,NaN
31,65B9EEA6E1CC6BB9F0CD2A47751A186F,NaN,NaN
33,A97DA629B098B75C294DFFDC3E463904,NaN,NaN
38,7F6FFAA6BB0B408017B62254211691B5,NaN,NaN
...,...,...,...
60016,3DC09677E0FDB539A31D497C4FB25F20,general practitioner,NaN
60017,39D96AC1450B2D517807DC8A94B26C17,Ophthalmology,NaN
60020,4473D870B5E31FAA40D2C45E1FF6DC27,NaN,NaN
60021,A4F554EB2C0934E7FDE2511E8C1573BA,NaN,NaN


A significant portion of missing data is attributed to the *mainspecialty* attribute. Because the OLAP application involves knowing the doctor's specialty. Doctors with missing specialties will be dropped. The *age* attribute in this case is not focused on. Duplicate doctor data will also be dropped.

In [27]:
df_doctors.dropna(subset=['mainspecialty'], inplace=True)
df_doctors.drop_duplicates(subset=['doctorid'], inplace=True)

In [28]:
df_doctors["mainspecialty"].unique()

array(['General Medicine', 'Family Medicine', 'Vascular Medicine', ...,
 'Bone alignment therapy', 'Healot Biomekaniks', 'Allergist'],
 dtype=object)

Unfortunately for this dataset, the gathering of doctor data in regard to their specialty was done poorly. This means inconsistent specialty of doctor. A strategy to somehow make the data a bit more consistent is to group specialties that are similar. Those that were not grouped will be analyzed so that it could either be dropped or processed to have consistent values with similar grouped values. 

In [29]:
# Group the values
grouped = df_doctors.groupby('mainspecialty')

# Find the unique values
unique_values = df_doctors['mainspecialty'].value_counts()

# Output the values that were not grouped (they appear only once in the column)
outliers = unique_values[unique_values == 1]

unique_values

Internal Medicine 3812
General Medicine 2317
Pediatrics 1709
Family Medicine 894
General Physician 881
 ... 
General Physician (Pediatrics and Adult) 1
Marc 1
Orthopaedics (Bone and Joint Surgery) 1
Generalists 1
Allergist 1
Name: mainspecialty, Length: 3974, dtype: int64

In [30]:
outliers

ORTHODONTIST/Veneers 1
General Medicine/Occupational Health Physcian 1
Adult Psychiatry and Behavioral Medicine 1
Faye Garciano 1
Child psychiatrist 1
 ..
General Physician (Pediatrics and Adult) 1
Marc 1
Orthopaedics (Bone and Joint Surgery) 1
Generalists 1
Allergist 1
Name: mainspecialty, Length: 3085, dtype: int64

Unfortunately, even if grouping helps in showing the outliers. There are some specialties that are valid and some that are not, and it would be unwise to carelessly drop these values as they might be important for analysis in the OLAP application. Thus, they will be left as is.

# Cleaning the appointments table
The clinics table has the following attributes:
- pxid
- clinicid
- doctorid
- apptid
- status
- TimeQueued
- QueueDate
- StartTime
- EndTime
- type
- Virtual
  
For the appointment data, let us ignore the attributes related to time as special cleaning is needed. The attribute to checks will be the validity of the appointment. Since the fact table is essential in the OLAP application, the appointments made contain necessary data such as *QueueDate, status, type, *and* virtual*. The key attribute must also be complete to fully connect the three dimensional tables.

In [31]:
columns_to_check = ["clinicid", "doctorid", "apptid", "pxid", "QueueDate", "status", "type", "Virtual"]
df_appointment = df_appointment.dropna(subset=columns_to_check)

Because all the query statements in the OLAP application will use the appointment table as a way to connect to the other connection table. We need to ensure that all key attributes do actually exist in the dimension tables. Thus, if a key value in an appointment table does not exist in the corresponding table, it will be dropped to ensure validity and integrity.

In [32]:
df_appointment = df_appointment[df_appointment['clinicid'].isin(df_clinics['clinicid'])]
df_appointment = df_appointment[df_appointment['doctorid'].isin(df_doctors['doctorid'])]
df_appointment = df_appointment[df_appointment['pxid'].isin(df_px['pxid'])]
df_appointment.drop_duplicates(subset=['apptid'], inplace=True)

,pxid,clinicid,doctorid,apptid,status,TimeQueued,QueueDate,StartTime,EndTime,type,Virtual
npartitions=1,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...


Similarly to the clinics table. the *Virtual* attribute will be converted into tinyInt 0 and 1 to ensure a smooth ETL process.

In [33]:
df_appointment['Virtual'] = df_appointment['Virtual'].map({'True': 1, 'False': 0})

Time-related attributes will simply be changed to the appropriate data type based on their name.

In [34]:
time_columns = ["TimeQueued", "QueueDate", "StartTime", "EndTime"]

for time in time_columns:
    df_appointment[time] = dd.to_datetime(df_appointment[time])
    if "Date" in time:
        df_appointment[time] = df_appointment[time].dt.date
    else:
        df_appointment[time] = df_appointment[time].dt.time

# Renaming
Finally, to follow database naming convention. All attributes except boolean attributes will follow the snake case naming convention. While boolean attributes will follow the camel case naming convention.

In [35]:
df_clinics = df_clinics.rename(columns={
    'clinicid': 'clinic_id',
    'hospitalname': 'hospital_name',
    'IsHospital': 'isHospital',
    'City': 'city',
    'Province': 'province',
    'RegionName': 'region_name'
})

df_px = df_px.rename(columns={
    'pxid' : 'px_id'
})

df_doctors = df_doctors.rename(columns={
    'doctorid': 'doctors_id',
    'mainspecialty': 'main_specialty'
})

df_appointment = df_appointment.rename(columns={
    'pxid': 'px_id',
    'clinicid': 'clinic_id',
    'doctorid': 'doctors_id',
    'apptid': 'appt_id',
    'TimeQueued': 'timequeued',
    'QueueDate': 'queuedate',
    'StartTime': 'starttime',
    'EndTime': 'endtime',
    'Virtual': 'isVirtual'
})


In [36]:
df_doctors.to_csv("cleaned_doctors.csv", index=False)
#For some reason, clinic.csv headers still defaults, so headers are explicitly added
df_clinics.to_csv("cleaned_clinics.csv", index=False, header=['clinic_id', 'hospital_name', 'isHospital', 'city', 'province', 'region_name'])
df_px.to_csv("cleaned_px.csv", index=False)
#Same with clinic.csv explicitly add the headers
df_appointment.to_csv("cleaned_appointments.csv", index=False, single_file=True, header=['px_id', 'clinic_id', 'doctors_id', 'appt_id', 'status', 'timequeued', 'queuedate', 'starttime', 'endtime', 'type', 'isVirtual'])

['/data/notebook_files/cleaned_appointments.csv']